In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Enable inline plotting
%matplotlib inline

In [ ]:
def generate_star(center=(0, 0), num_points=5, outer_radius=1, inner_radius=0.5, homogeneous=False):
    '''
    '''
    angles = np.linspace(np.pi/2, 5*np.pi/2, num_points*2, endpoint=False)
    radii = np.array([outer_radius, inner_radius] * num_points)
    x = radii * np.cos(angles) + center[0]
    y = radii * np.sin(angles) + center[1]
    return np.column_stack((x, y)) if not homogeneous else np.column_stack((x, y, np.ones(num_points*2)))

def plot_star(ax, star, title, global_view):
    ax.plot(star[:, 0], star[:, 1], 'b-')
    ax.plot([star[-1, 0], star[0, 0]], [star[-1, 1], star[0, 1]], 'b-')  # Connect last to first
    ax.set_title(title)
    ax.axis('equal')
    if global_view:
        ax.set_xlim(-10, 10)
        ax.set_ylim(-10, 10)
        ax.axhline(0, color='black')
        ax.axvline(0, color='black')
    ax.grid(True)
    # Arrow from star center to first vertex
    star_center = np.mean(star, axis=0)
    vector_to_point = star[0] - star_center
    direction = vector_to_point / np.linalg.norm(vector_to_point)
    arrow_length = np.linalg.norm(vector_to_point) / 2
    arrow_end = star_center + arrow_length * direction
    headwidth = 6 if global_view else 8
    width = 1 if global_view else 2
    ax.annotate('', xy=arrow_end, xytext=star_center,
                arrowprops=dict(facecolor='green', shrink=0.05, width=width, headwidth=headwidth))

def plot_stars(stars, global_view=False):
    if len(stars) == 1 or not isinstance(stars, list):
        print("There must be a list of multiple stars to plot")
        return
    num_stars = len(stars)
    fig, axs = plt.subplots(1, num_stars, figsize=(10, 5))
    for i, star in enumerate(stars):
        plot_star(axs[i], star[:, :2], f'Star {i+1}', global_view)

# 2D homogeneous transform helpers
def rotate(angle_degrees):
    a = np.radians(angle_degrees)
    c, s = np.cos(a), np.sin(a)
    return np.array([[c, -s, 0],
                     [s,  c, 0],
                     [0,  0, 1]])

def scale(sx, sy):
    return np.array([[sx, 0,  0],
                     [0,  sy, 0],
                     [0,  0,  1]])

def translate(tx, ty):
    return np.array([[1, 0, tx],
                     [0, 1, ty],
                     [0, 0, 1]])

def to_h(points):
    # to homogeneous
    return np.hstack([points, np.ones((points.shape[0], 1))])

def from_h(points_h):
    # from homogeneous
    return points_h[:, :2] / points_h[:, 2, np.newaxis]

In [ ]:
# Image 1: Star2 is a translation of Star1
star1 = generate_star(center=(0, 0), outer_radius=1, inner_radius=0.5)
T = translate(3, 2)                     # single combined matrix (only a translation here)
star2 = from_h(to_h(star1) @ T.T)
plot_stars([star1, star2])

In [ ]:
# Image 2: Star2 is a pure rotation of Star1 (about origin/its center, since center is (0,0))
star1 = generate_star(center=(0, 0), outer_radius=1, inner_radius=0.5)
R = rotate(90)                           # +90° => arrow points left
star2 = from_h(to_h(star1) @ R.T)
plot_stars([star1, star2])

In [ ]:
# Image 3: Star1 centered away from origin; Star2 = rotate a bit and translate
star1 = generate_star(center=(2, 2), outer_radius=1, inner_radius=0.5)
# Build a single combined matrix for: small rotation about the star's center and a slight translation
cx, cy = 2, 2
M = translate(cx, cy) @ rotate(30) @ translate(-cx, -cy) @ translate(0.2, 0.1)
star2 = from_h(to_h(star1) @ M.T)
plot_stars([star1, star2])

In [ ]:
# Image 4: Star2 is rotated 180° and moved to the left
star1 = generate_star(center=(2, 0), outer_radius=1, inner_radius=0.5)
# rotate about origin and translate to ~(-1, 0.6)
M = rotate(180) @ translate(-3, 0.6)     # single matrix product
star2 = from_h(to_h(star1) @ M.T)
plot_stars([star1, star2])

In [ ]:
# Image 5: Global view. Star1 small near (0,2). Star2 mirrored vertically and moved below.
star1 = generate_star(center=(0, 2), outer_radius=1.1, inner_radius=0.55)
# reflect in x (scale y by -1) about origin, then translate down
M = scale(1, -1) @ translate(0, -5)
star2 = from_h(to_h(star1) @ M.T)
plot_stars([star1, star2], global_view=True)

In [ ]:
# Image 6: Global view. Make Star2 larger, rotated, and shifted down-right.
star1 = generate_star(center=(4, 2), outer_radius=1.0, inner_radius=0.5)
# scale 2x about the star's center, rotate a little, then translate down-left
cx, cy = 4, 2
M = translate(cx, cy) @ scale(2, 2) @ rotate(20) @ translate(-cx, -cy) @ translate(-3.5, -6)
star2 = from_h(to_h(star1) @ M.T)
plot_stars([star1, star2], global_view=True)